# Scratch File for Shawn. No one else should edit or check in.

In [2]:
%load_ext autoreload
%autoreload 2
import shared_utils as su
import numpy as np
import pandas as pd

In [ ]:
trip_data = su.read_trip_data()
trip_data.head()

In [174]:
station_data = su.read_station_data()

Basing anything on a customers zip code seems to require some cleaning up of that field. Rather than replace the field values I've opted to create a new column with scrubbed data.

In [80]:
su.clean_zip_codes(trip_data)

In [77]:
# Counts of how often a customer is from a san francisco zip code
sf_only_customer_trip_data = trip_data.groupby("Clean_Zip_Code").filter(lambda x: x.name in su.SF_ZIP_CODES)
sf_only_customer_trip_data = trip_data[trip_data.Clean_Zip_Code.map(lambda x : x in su.SF_ZIP_CODES)]
grouped_by_zip_code = sf_only_customer_trip_data.groupby("Clean_Zip_Code")

In [84]:
grouped_by_zip_code.count().sort_values("Trip_ID", ascending=False)

,Trip_ID,Duration,Start_Date,Start_Station,Start_Terminal,End_Date,End_Station,End_Terminal,Bike_#,Subscriber_Type,Zip_Code
Clean_Zip_Code,,,,,,,,,,,
94107,46629,46629,46629,46629,46629,46629,46629,46629,46629,46629,46629
94105,20311,20311,20311,20311,20311,20311,20311,20311,20311,20311,20311
94133,16246,16246,16246,16246,16246,16246,16246,16246,16246,16246,16246
94103,14983,14983,14983,14983,14983,14983,14983,14983,14983,14983,14983
94111,10960,10960,10960,10960,10960,10960,10960,10960,10960,10960,10960
94102,10150,10150,10150,10150,10150,10150,10150,10150,10150,10150,10150
94109,6413,6413,6413,6413,6413,6413,6413,6413,6413,6413,6413
94158,4071,4071,4071,4071,4071,4071,4071,4071,4071,4071,4071
94117,4070,4070,4070,4070,4070,4070,4070,4070,4070,4070,4070


In [101]:
zip_df = su.read_zip_code_data()

In [118]:
zip_df[(zip_df.latitude == 37.79) & (zip_df.longitude == -122.39) & (zip_df.type=='STANDARD')]

,zip,type,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,latitude,longitude,world_region,country,decommissioned,estimated_population,notes
39607,94111,STANDARD,San Francisco,NaN,NaN,CA,San Francisco County,America/Los_Angeles,415510650,37.79,-122.39,NaN,US,0,3905,NaN


In [176]:
# wanted to get zip codes based of long and lat, but the us postal service data either didn't have entries for every 
# one of our values or had too many entries for each value
#station_data["Zip_Code"] = [""] * len(station_data)

def proccess_station_row(row):  
    df = zip_df[(zip_df.latitude == round(row.lat,2)) & (zip_df.longitude == round(row.long, 2)) & (zip_df.type=='STANDARD')]
    if len(df) == 1:
        result = df.zip.iloc[0]
    else:
        result = np.NaN
    return result

station_data.apply(proccess_station_row, axis=1)


0       NaN
1     95113
2     95113
3     95113
4       NaN
5     95113
6     95113
7       NaN
8       NaN
9       NaN
10      NaN
11      NaN
12      NaN
13      NaN
14      NaN
15      NaN
16      NaN
17      NaN
18      NaN
19      NaN
20      NaN
21      NaN
22      NaN
23      NaN
24      NaN
25      NaN
26      NaN
27      NaN
28      NaN
29      NaN
      ...  
40    94111
41      NaN
42    94111
43    94111
44      NaN
45      NaN
46      NaN
47      NaN
48      NaN
49    94105
50      NaN
51      NaN
52    94105
53      NaN
54      NaN
55      NaN
56      NaN
57      NaN
58      NaN
59      NaN
60      NaN
61    94133
62    94111
63      NaN
64      NaN
65      NaN
66      NaN
67      NaN
68      NaN
69    95110
dtype: float64

In [175]:
station_data[station_data.landmark == "San Francisco"]

,station_id,name,lat,long,dockcount,landmark,installation,Zip_Code
32,41,Clay at Battery,37.795001,-122.399970,15,San Francisco,2013-08-19,94111
33,42,Davis at Jackson,37.797280,-122.398436,15,San Francisco,2013-08-19,94111
34,45,Commercial at Montgomery,37.794231,-122.402923,15,San Francisco,2013-08-19,94111
35,46,Washington at Kearney,37.795425,-122.404767,15,San Francisco,2013-08-19,94108
36,47,Post at Kearney,37.788975,-122.403452,19,San Francisco,2013-08-19,94108
37,48,Embarcadero at Vallejo,37.799953,-122.398525,15,San Francisco,2013-08-19,94111
38,49,Spear at Folsom,37.790302,-122.390637,19,San Francisco,2013-08-20,94105
39,50,Harry Bridges Plaza (Ferry Building),37.795392,-122.394203,23,San Francisco,2013-08-20,94105
40,51,Embarcadero at Folsom,37.791464,-122.391034,19,San Francisco,2013-08-20,94105
41,39,Powell Street BART,37.783871,-122.408433,19,San Francisco,2013-08-25,94102
